# Unsupervised Land Use/Land Cover with K-Means

This notebook runs an unsupervised LULC classification on a stacked Sentinel-2 raster using k-means.

Highlights:
- Uses cloud/shadow masking via SCL/QA60.
- Adds NDVI/NDWI/NDBI to improve class separability.
- Fits k-means on a sampled subset (memory-safe) and predicts in tiles.
- Writes a labeled GeoTIFF (255 for masked/invalid).

## 1) Setup
Install dependencies if needed: `rasterio`, `numpy`, `scikit-learn`.
Set the input stack path and output label path. The stack should contain bands B02,B03,B04,B08 and an SCL/QA60 mask band.

In [ ]:
!pip install rasterio scikit-learn numpy


In [ ]:

import numpy as np
import rasterio
from rasterio.windows import Window
from sklearn.cluster import MiniBatchKMeans

# --- Config ---
STACK_TIF = "s2_stack.tif"       # Input: stacked bands + mask (e.g., SCL or QA60 as last band)
MASK_BAND = "SCL"                # "SCL" or "QA60" if last band is a mask; set None to disable masking
N_CLUSTERS = 6                   # Tune to expected classes
SAMPLE_SIZE = 200_000            # Pixels to fit k-means (subsample for speed/memory)
TILE = 1024                      # Tile size for prediction to keep memory low
OUT = "kmeans_labels.tif"       # Output: labeled raster (uint8; 255 = masked)


## 2) Helper Functions
Mask clouds/shadows, add spectral indices, and prepare features.

In [ ]:
def build_mask(stack):
    """Return boolean mask of valid pixels based on SCL/QA60 in last band."""
    if MASK_BAND is None:
        return np.ones(stack.shape[1:], dtype=bool)
    scl = stack[-1]
    # Keep: 4 veg, 5 bare soil, 6 water; drop clouds/shadows/haze
    keep = np.isin(scl, [4, 5, 6])
    return keep


def add_indices(arr):
    """Compute NDVI/NDWI/NDBI and stack with inputs.
    Expects arr with bands [B02,B03,B04,B08,...]; if SWIR not present, NDBI falls back to red."""
    b2, b3, b4, b8 = arr[0], arr[1], arr[2], arr[3]
    ndvi = (b8 - b4) / np.clip(b8 + b4, 1e-6, None)
    ndwi = (b3 - b8) / np.clip(b3 + b8, 1e-6, None)
    # If a SWIR band is present after B08, use it; else fall back to red
    b_swir = arr[4] if arr.shape[0] > 4 else b4
    ndbi = (b_swir - b8) / np.clip(b_swir + b8, 1e-6, None)
    return np.stack([ndvi, ndwi, ndbi], axis=0)


## 3) Fit K-Means on a Sampled Subset
Reads the full stack once, masks clouds/shadows, adds indices, and fits k-means on a sampled subset to stay memory-safe.

In [ ]:
with rasterio.open(STACK_TIF) as src:
    full = src.read(list(range(1, src.count)))  # (bands, H, W)

mask = build_mask(full)
features = full[:-1] if MASK_BAND else full  # drop mask band if present
indices = add_indices(features)
feat_stack = np.concatenate([features, indices], axis=0)  # (F, H, W)

X = feat_stack[:, mask].T  # (N, F)
if X.shape[0] > SAMPLE_SIZE:
    idx = np.random.choice(X.shape[0], SAMPLE_SIZE, replace=False)
    X_fit = X[idx]
else:
    X_fit = X

kmeans = MiniBatchKMeans(
    n_clusters=N_CLUSTERS,
    batch_size=2048,
    n_init=5,
    max_iter=100,
).fit(X_fit)
print("Fitted k-means on", X_fit.shape[0], "pixels with", feat_stack.shape[0], "features")


## 4) Tiled Prediction and Write Labeled GeoTIFF
Predicts labels in tiles to avoid high memory use. Masked pixels get value 255.

In [ ]:
with rasterio.open(STACK_TIF) as src:
    meta = src.meta.copy()
    meta.update(count=1, dtype="uint8")
    H, W = src.height, src.width

    with rasterio.open(OUT, "w", **meta) as dst:
        for r0 in range(0, H, TILE):
            for c0 in range(0, W, TILE):
                h = min(TILE, H - r0)
                w = min(TILE, W - c0)
                window = Window(c0, r0, w, h)
                arr = src.read(window=window)
                m = build_mask(arr)
                feats = arr[:-1] if MASK_BAND else arr
                idxs = add_indices(feats)
                feat_tile = np.concatenate([feats, idxs], axis=0)
                flat = feat_tile.reshape(feat_tile.shape[0], -1).T  # (N, F)
                labels = np.full(flat.shape[0], 255, dtype=np.uint8)
                valid = m.ravel()
                if valid.any():
                    labels[valid] = kmeans.predict(flat[valid]).astype(np.uint8)
                labels = labels.reshape(h, w)
                dst.write(labels, 1, window=window)

print(f"Wrote labels to {OUT}")


## 5) Quick-Look Visualization (Optional)
Preview the label map and a stretched RGB from the same stack (needs matplotlib).

In [ ]:
import matplotlib.pyplot as plt

with rasterio.open(OUT) as src:
    labels = src.read(1)
with rasterio.open(STACK_TIF) as src:
    rgb = src.read([3, 2, 1])  # B04,B03,B02

def stretch(img):
    img = np.stack([img[i] for i in range(img.shape[0])], axis=-1)
    lo, hi = np.percentile(img, 2), np.percentile(img, 98)
    return np.clip((img - lo) / (hi - lo + 1e-6), 0, 1)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.imshow(stretch(rgb))
plt.title("RGB (stretched)")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(labels, cmap="tab20", vmin=0, vmax=N_CLUSTERS)
plt.title("K-Means Labels (255 = masked)")
plt.axis("off")
plt.tight_layout()
plt.show()


## Notes and Next Steps
- Adjust `N_CLUSTERS` to match expected land-use types; inspect centroids to assign semantic labels.
- Ensure the stack includes a valid SCL/QA60 band for masking; otherwise set `MASK_BAND = None`.
- For very large areas, reduce `TILE` or increase `SAMPLE_SIZE` for better fitting.
- Add more indices (e.g., MNDWI, EVI) if SWIR/Red-Edge bands are available.